In [1]:
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, UpSampling2D, Input
from keras.models import Model, Sequential
from keras.utils import img_to_array, to_categorical
from keras import optimizers
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from os import listdir

# Data loading and preprocessing
root_dir = "D:\\FINAL YEAR PROJECT\\senior project code\\FinalProject\\datasets\\NEW2"

In [2]:
# Function to convert an image into a tensor
def convert_img_to_tensor(fpath):
    img = cv2.imread(fpath)
    img = cv2.resize(img, (256, 256))
    return img_to_array(img)

# Load dataset and label data
def get_img_data_and_label(root_dir):
    image_dataset = []
    image_label = []
    classes = listdir(root_dir)
    class_mapping = {c: i for i, c in enumerate(classes)}

    for cls in classes:
        skin_img_list = listdir(f"{root_dir}/{cls}")
        for imgfile in skin_img_list:
            filepath = f"{root_dir}/{cls}/{imgfile}"
            image_dataset.append(convert_img_to_tensor(filepath))
            image_label.append(class_mapping[cls])
    
    return image_dataset, image_label, len(classes), classes

In [3]:
image_dataset, image_labels, no_of_output_layer, classes = get_img_data_and_label(root_dir)

In [4]:
# Normalize and reshape data
image_dataset = np.array(image_dataset, dtype=np.float32) / 255.0
image_dataset = image_dataset.reshape(-1, 256, 256, 3)

In [15]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape, UpSampling2D

input_shape = (256, 256, 3)

autoencoder = Sequential([
    # Encoder
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    MaxPooling2D((2, 2), padding='same'),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),

    # Encoder to bottleneck
    Flatten(),
    Dense(64, activation='relu'),  # Bottleneck layer
])

# Decoder
autoencoder.add(Dense(128 * 16 * 16, activation='relu'))  # Restore to the right dimension for reshaping
autoencoder.add(Reshape((16, 16, 128)))

# Upsample to restore original shape
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(64, (3, 3), activation='relu', padding='same'))

autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(32, (3, 3), activation='relu', padding='same'))

# This last step restores the output to the original input shape
autoencoder.add(Conv2D(3, (3, 3), activation='sigmoid', padding='same'))

# Compile and train
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [16]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
# Train the autoencoder
autoencoder.fit(image_dataset, image_dataset, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 816s 24s/step - accuracy: 0.2326 - loss: 0.5332 - val_accuracy: 0.0676 - val_loss: 0.2362
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 657s 20s/step - accuracy: 0.2224 - loss: 0.4195 - val_accuracy: 0.1920 - val_loss: 0.1433
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 244s 7s/step - accuracy: 0.4171 - loss: 0.3783 - val_accuracy: 0.4016 - val_loss: 0.1503
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 251s 7s/step - accuracy: 0.4766 - loss: 0.3499 - val_accuracy: 0.7927 - val_loss: 0.1195
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 248s 7s/step - accuracy: 0.6049 - loss: 0.3297 - val_accuracy: 0.6491 - val_loss: 0.1177
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 237s 7s/step - accuracy: 0.5761 - loss: 0.3245 - val_accuracy: 0.1281 - val_loss: 0.1130
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 242s 7s/step - accuracy: 0.4777 - loss: 0.3203 - val_accuracy: 0.8734 - val_loss: 0.1063
Epoch 8/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 249s 7s/step - accuracy: 0.6182 - loss: 0.3258 - val_accuracy: 0.6734 -

In [3]:
# Extract encoder model
encoder_model = Model(autoencoder.input, autoencoder.layers[8].output)  # The bottleneck layer is the eighth layer

NameError: name 'autoencoder' is not defined